In [3]:
import findspark
findspark.init('/opt/spark/')
findspark.find()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("K_mean").getOrCreate()

df = spark.read.csv("uber-raw-data-aug14.csv",header=True)





df.show()

df = spark.read.csv("uber-raw-data-aug14.csv", inferSchema=True, header=True)
df.groupBy("Base").count().show()
df.groupBy("Date/Time").count().show()
df.printSchema()
from pyspark.ml.feature import StringIndexer


indexer=StringIndexer(inputCol='Base',outputCol='Base'+"_N")
df=indexer.fit(df).transform(df)
df.show()
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
df.columns
feat_cols = ['Lat', 'Lon','Base_N']
assembler = VectorAssembler(inputCols=feat_cols, outputCol='features')
final_df = assembler.transform(df)
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features',outputCol='scaled_feat',withStd=True,withMean = False)
scaled_model =scaler.fit(final_df)
cluster_df= scaled_model.transform(final_df)
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

eval = ClusteringEvaluator(predictionCol="prediction",featuresCol="scaled_feat",metricName="silhouette",distanceMeasure="squaredEuclidean")


silhouette_score =[]




for k in range(2,11):
    kmeans_algo = KMeans(featuresCol='scaled_feat',k=k)
    kmeans_fit = kmeans_algo.fit(cluster_df)
    output = kmeans_fit.transform(cluster_df)
    score=eval.evaluate(output)
    silhouette_score.append(score)
    print(f"k{k}\t{(score)}\t")



training_data, test_data = cluster_df.randomSplit([0.8, 0.2], seed=70)

kmeans = KMeans().setK(3).setSeed(20)
model = kmeans.fit(training_data)


from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(3).setSeed(20)
model = kmeans.fit(training_data)
model.save("kmeans_model_k5.model")



+----------------+-------+--------+------+
|       Date/Time|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|8/1/2014 0:03:00|40.7366|-73.9906|B02512|
|8/1/2014 0:09:00| 40.726|-73.9918|B02512|
|8/1/2014 0:12:00|40.7209|-74.0507|B02512|
|8/1/2014 0:12:00|40.7387|-73.9856|B02512|
|8/1/2014 0:12:00|40.7323|-74.0077|B02512|
|8/1/2014 0:13:00|40.7349|-74.0033|B02512|
|8/1/2014 0:15:00|40.7279|-73.9542|B02512|
|8/1/2014 0:17:00| 40.721|-73.9937|B02512|
|8/1/2014 0:19:00|40.7195| -74.006|B02512|
|8/1/2014 0:20:00|40.7448|-73.9799|B02512|
|8/1/2014 0:21:00|40.7399|-74.0057|B02512|
|8/1/2014 0:25:00|40.7651|-73.9683|B02512|
|8/1/2014 0:27:00|40.7354|-74.0081|B02512|
|8/1/2014 0:29:00|40.7339|-74.0028|B02512|
|8/1/2014 0:29:00|40.7364|-74.0301|B02512|
|8/1/2014 0:29:00|40.7364|-74.0301|B02512|
|8/1/2014 0:30:00|40.7252|-73.9516|B02512|
|8/1/2014 0:30:00|40.7433| -73.986|B02512|
|8/1/2014 0:34:00|40.7437|-73.9884|B02512|
|8/1/2014 0:36:00|40.7406|-74.0077|B02512|
+----------

k2	0.47938024903641385	


k3	0.5464672423620176	


k4	0.4344781690508035	


k5	0.40582608080378774	


k6	0.3805012106587415	


k7	0.4583674926980751	


k8	0.39482754966466754	


k9	0.43942988991675513	


k10	0.5172072705359402	


In [4]:
from kafka import KafkaProducer, KafkaConsumer
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

# create a Kafka producer to produce test data
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

In [5]:
from pyspark.sql.functions import to_json, struct
from kafka import KafkaProducer
import json
from time import sleep

# create a Kafka producer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], 
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

# convert the test_data to a JSON string
test_data_json = test_data.select(to_json(struct([test_data[x] for x in test_data.columns])).alias("value")).collect()

for data in test_data_json:
    producer.send('test_topic', value=data['value'])
    sleep(1)
    print(f"{data['value']}")

{"Date/Time":"8/1/2014 0:00:00","Lat":40.729,"Lon":-73.9422,"Base":"B02598","Base_N":1.0,"features":{"type":1,"values":[40.729,-73.9422,1.0]},"scaled_feat":{"type":1,"values":[933.5505454380506,-1202.6499472319315,0.9076564359470978]}}
{"Date/Time":"8/1/2014 0:00:00","Lat":40.7424,"Lon":-74.0044,"Base":"B02598","Base_N":1.0,"features":{"type":1,"values":[40.7424,-74.0044,1.0]},"scaled_feat":{"type":1,"values":[933.8576872119431,-1203.6616134620115,0.9076564359470978]}}
{"Date/Time":"8/1/2014 0:01:00","Lat":40.6906,"Lon":-73.9933,"Base":"B02598","Base_N":1.0,"features":{"type":1,"values":[40.6906,-73.9933,1.0]},"scaled_feat":{"type":1,"values":[932.6703779665974,-1203.4810749547144,0.9076564359470978]}}
{"Date/Time":"8/1/2014 0:01:00","Lat":40.7052,"Lon":-74.0094,"Base":"B02598","Base_N":1.0,"features":{"type":1,"values":[40.7052,-74.0094,1.0]},"scaled_feat":{"type":1,"values":[933.0050249739727,-1203.7429371139472,0.9076564359470978]}}
{"Date/Time":"8/1/2014 0:01:00","Lat":40.7403,"Lon

KeyboardInterrupt: 

In [6]:

    
from kafka import KafkaConsumer
import json
from pyspark.ml.clustering import KMeansModel
import time

# load the trained KMeans model from file/directory
loaded_model = KMeansModel.load("kmeans_model_k3.model")

# create a Kafka consumer
consumer = KafkaConsumer('test_topic', bootstrap_servers=['localhost:9092'],
                         auto_offset_reset='earliest', 
                         enable_auto_commit=True, 
                         value_deserializer=lambda m: json.loads(m.decode('utf-8')))

for message in consumer:
    
    data = message.value
    json_object = json.loads(data)
    df = spark.createDataFrame([(json_object['Lat'], json_object['Lon'],json_object['Base_N'])], ['Lat', 'Lon','Base_N'])
    
    # assemble the feature vectors
    assembler = VectorAssembler(inputCols=["Lat", "Lon","Base_N"], outputCol="features")
    indexed = assembler.transform(df).select("features")

    predictions = loaded_model.transform(indexed)
    
    predictions.show()
    


+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[40.729,-73.9422,...|         2|
+--------------------+----------+

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[40.7424,-74.0044...|         2|
+--------------------+----------+

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[40.6906,-73.9933...|         2|
+--------------------+----------+

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[40.7052,-74.0094...|         2|
+--------------------+----------+

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[40.7403,-74.0081...|         2|
+--------------------+----------+

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[40.7686,-73.9697...|         2|
+--------

KeyboardInterrupt: 